# Assignment 2
Perform a similar operation to what described above, this time for Master students. Notice that this data is more tricky, as there are many missing records in the IS-Academia database. Therefore, try to guess how much time a master student spent at EPFL by at least checking the distance in months between `Master semestre 1` and `Master semestre 2`. If the Mineur field is not empty, the student should also appear registered in `Master semestre 3`. Last but not the least, don't forget to check if the student has an entry also in the `Projet Master` tables. Once you can handle well this data, compute the "average stay at EPFL" for master students. Now extract all the students with a `Spécialisation` and compute the "average stay" per each category of that attribute -- compared to the general average, can you find any specialization for which the difference in average is statistically significant?

Useful usual imports at first

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import html5lib 
from lxml import html
import numpy as np
from helpers import *
#import helpers as helpers

Extracting the master students in the similar fashion we did in the first assignment : we associate each year/master stage with its value, that we extract from the html page using the BeautifulSoup package.

In [ ]:
# The URL of the "start" of the IS-Academia page listing the students.
home_url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_x_GPS=-1&ww_i_reportModel=133685247"

# The general form of the URL, with the fields to be replaced later (e.g. [UNITE_ACADEMIQUE])
base_url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=[UNITE_ACADEMIQUE]&ww_x_PERIODE_ACAD=[PERIODE_ACADEMIQUE]&ww_x_PERIODE_PEDAGO=[PERIODE_PEDAGOGIQUE]&ww_x_HIVERETE=null"

# Get the raw content from the page
with urllib.request.urlopen(home_url) as url:
    s = url.read()

soup = BeautifulSoup(s, 'html.parser')

# Parse the content
url_section = 0
url_years = {}
url_master = {}

#For loop on all the possible fields with <option> to get the value associated with each year, or each "Bachelor" semester
for link in soup.find_all('option'):
    if link.text == 'Informatique':
        url_section = link.get('value')
    if "20" in link.text:
        url_years[link.text] = link.get('value')
    if 'Master' in link.text or 'Mineur' in link.text:
        url_master[link.text] = link.get('value')

        
print(url_years)
print(url_master)

Function that replaces the general fields in the base_url with the fields for a given year and level of study

In [ ]:
def getFullUrl(PeriodeAcad, PeriodePedag):
    """
    Given an academic period (years) and a pedagogic period (bachelor semester), 
    fills the general URL with the values associated to those parameters.
    
    @param PeriodeAcad :  Academic year string, in the form ('2015-2016')
    @param PeriodePedag : Pedagogic period string, in the form ('Bachelor x')
    @return url :         The complete url associated to those period, from which we can fetch the students.
    
    """
    url = base_url
    url = url.replace('[UNITE_ACADEMIQUE]', str(url_section))
    url = url.replace('[PERIODE_ACADEMIQUE]', str(url_years[PeriodeAcad]))
    url = url.replace('[PERIODE_PEDAGOGIQUE]', str(url_master[PeriodePedag]))
    return url

We noticed that in the way the EPFL master works, we will have all students that start in the autumn semester in `MS1` and then moves onto the `MS2`, then every subsqeuent autumn semester that he does will be counted into `MS3`.

The way our data structure extracts the information we want from the table of students is as simple as possible. Keeping track of a student and noting each semester in which is present can be quite tricky, even very difficult, due to the holes into the IS-Academia data. The simplest way we found to address this problem is not trying to make any assumptions into where a student that disappears goes. We count each time a student appears in ones of the master semesters or master project semesters and add a $1$ each time we find an occurence of the same student. Hence its master duration will be the sum of the semesters in which the student appears.

In [ ]:
def MasterStart(year_string,master_student,pedag) : 
    """
    Given a year and a pedagogic period, edits the master_student frame, adding the students 
    that are not present and editting the values for the ones present (adding a +1 for the pedagogic period)
    
    @param year_string :    Academic year string, in the form ('2015-2016')
    @param master_student : DataFrame containing the occurrence of each students into any possible semester of the master.
    @param pedag :          Pedagogic period string (e.g. in the form 'Master semestre x')
    @return master_student: The editted version of our DataFrame.
    
    """    
    #Reading the HTML page
    current_url = getFullUrl(year_string, pedag)
    
    with urllib.request.urlopen(current_url) as url:
        html = url.read()
    soup = BeautifulSoup(html, 'html.parser')
    master_frame = pd.read_html(soup.prettify(), header=1)
    
    #If the page has actually some students in it.
    if  len(master_frame)>0 :
        master_frame = master_frame[0].drop(0,axis=0)
        master_frame_sciper = master_frame[['No Sciper']]
        master_frame = master_frame.set_index('No Sciper')
        for sciper in master_frame_sciper.values:
            sciper = sciper[0]
            
            #As a student can be in 'Master semestre 1' only once, we add all of them automatically.
            if pedag == 'Master semestre 1' or sciper not in master_student.index:
                master_student.loc[sciper] = 0   
                master_student.loc[sciper,pedag] = 1
            else :
                master_student.loc[sciper,pedag] += 1   
            master_student.loc[sciper,'Civilité'] = master_frame.loc[sciper,'Civilité']
            master_student.loc[sciper,'Nom Prénom'] = master_frame.loc[sciper,'Nom Prénom']
            master_student.loc[sciper,'Spécialisation'] = master_frame.loc[sciper,'Spécialisation']
            master_student.loc[sciper,'Mineur'] = master_frame.loc[sciper,'Mineur']
    return master_student

We perform here the extraction from all the years and master stages for each student we find, going from the period $2007$ to $2016$, counting for each student. The following essentially calls the `MasterStart` function at each iteration.

In [ ]:
master_student = pd.DataFrame(columns = ['Civilité', 'Nom Prénom', 
                                         'Master semestre 1', 'Master semestre 2', 'Master semestre 3',
                                         'Projet Master printemps', 'Projet Master automne', 
                                         'Spécialisation', 'Mineur'])
master_student.index.name = 'No Sciper'

pedagogic_year = ['Master semestre 1','Master semestre 2','Master semestre 3',
                  'Projet Master printemps','Projet Master automne']
for year in range(2007,2016):
    year_string = str(year) + '-' + str(year+1)
    for pedag in  pedagogic_year :
         master_student = MasterStart(year_string,master_student,pedag)

**Assumptions**  : 
 - It is possible to have project master in spring and autumn (not really an assumption, but we realise it's possible to fail the master project and redo it, so the data is valid).
 - If no `projet master` field has been found, we assume that the student had his project outside epfl and add $+1$ semester to the total duration of his master.
 - If there are no entries in `Master semester 1` and `Master semester 2`, we also add a $+1$ to the total duration of the master, as a student needs to have done at least one of those, when he starts.

In [ ]:
print(master_student.index.is_unique)
## sum over all the master 1,2,3 plus additional master project.
## The two long lines starting with one add a semester to the master for students having :
## 0 semesters in MS1, MS2 // 0 semesters in Master project autumn/spring
time = (master_student['Master semestre 1']+master_student['Master semestre 2']+
        1-((master_student['Master semestre 1']+ master_student['Master semestre 2'])>=1) +
        master_student['Master semestre 3']+master_student['Projet Master printemps']+
        master_student['Projet Master automne']+
        1-((master_student['Projet Master printemps']+ master_student['Projet Master automne'])>=1)) 
master_student['Time to Finish']= time
master_student.sort_values(ascending=[True,True],by=['Master semestre 1', 'Master semestre 2']).head(10)

We now perform the statistical test to determine whether doing a specialisation effectively has an effect on the duration of the master. We chose a 1-sample t-test, as we want to compare if the difference from the mean duration of the master without any specialisation is statistically significant. Our null hypothesis is then that the two samples are from the ame distribution.

In [ ]:
import scipy.stats as stats
specialisation = master_student[master_student['Spécialisation']==master_student['Spécialisation']]                        
mean_time_specialisation = specialisation['Time to Finish']
mean_time  = master_student['Time to Finish'].mean()
print(mean_time)
print(mean_time_specialisation.mean())
stats.ttest_1samp(a= mean_time_specialisation, 
                 popmean= mean_time)

The result shows VERY clearly that the difference is meaningful, as the p-value is almost zero, rejecting our null hypothesis. We can see even by eye from the averages that we have at least a 0.5 longer master duration for the people doing a specialisation.

We will then compute the mean duration of the master for each of the different specialisations.

In [ ]:
master_student_grouped = master_student.groupby(master_student.Spécialisation)
mean_spec = pd.DataFrame(columns=['Mean'])

for spec,student in master_student_grouped:
    #print('spec', spec)
    #print('student', student['Time to Finish'])
    mean_spec.loc[spec] = np.mean(student['Time to Finish'].values)

mean_spec

In [ ]:
#print(master_student.head(20))
#master_student = master_student.drop('Mineur', axis=1)

master_student_with_Spec = master_student[master_student.Spécialisation == master_student.Spécialisation]
list_Spec = master_student_with_Spec['Spécialisation'].drop_duplicates().values

#print(master_student_with_Spec.shape)
#print(master_student_with_Spec.sort_index().head())

master_student_with_Spec['No Sciper'] = master_student_with_Spec.index
master_student_with_Spec = master_student_with_Spec.set_index(['Spécialisation'])
master_student_with_Spec = master_student_with_Spec.sort_index()
master_student_with_Spec = master_student_with_Spec.set_index([master_student_with_Spec.index, 'No Sciper'])

master_student_with_Spec.head(10)

#print(master_student_with_Spec.head())


In [ ]:
#segments.groupby(segments.name).seg_length.sum().sort_values(ascending=False, inplace=False).head(20)

#master_student_with_Spec.groupby(master_student_with_Spec.index).sum().head(10)

In [ ]:
#master_student['Spécialisation'].drop_duplicates()
print(list_Spec)
master_student_with_Spec.loc[list_Spec[1]]

Now we perform the statistical test on the students regrouped by their specialisation

In [ ]:
for spec in list_Spec:
    master_student_in_spec = master_student_with_Spec.loc[spec]
    print('Specialisation ' + spec)
    print(master_student_in_spec.loc[:,'Time to Finish'].shape)
    print(stats.ttest_1samp(a= np.array(master_student_in_spec.loc[:,'Time to Finish']), popmean=mean_time))
    print()
    
# Note : we get a NaN pvalue when there is only 1 student in a Specialisation as there is not enough data to perform
# a statistical test.

We observe that there is a statistically significant difference of Master duration for students choosing Specifications **Internet Computing**, **Foundations of Software**, **Signals, Images and Interfaces**, **Computer Engineering - SP**, **Computer Engineering - SP** and **Data Analytics**

# Assigment 3
**BONUS**: perform the gender-based study also on the Master students, as explained in 1. Use scatterplots to visually identify changes over time. Plot males and females with different colors -- can you spot different trends that match the results of your statistical tests?

In [ ]:
men_master_student = master_student.loc[master_student['Civilité']=='Monsieur']
women_master_student = master_student.loc[master_student['Civilité']=='Madame']

#print(men_master_student.head())

# Perform the statistical test
stats.ttest_ind(a= np.array(women_master_student.loc[:,'Time to Finish']),
                b= np.array(men_master_student.loc[:,'Time to Finish']),
                equal_var=False)    # Assume samples have equal variance?

In [ ]:
import matplotlib.pyplot as plt

x = np.random.normal(1, 2, 5000)
y = np.random.normal(-1, 3, 5000)
data = np.vstack([x, y]).T
bins = np.linspace(-10, 10, 30)

print(x.shape)
print(y.shape)

plt.hist(data, bins, alpha=0.7, label=['x', 'y'])
plt.legend(loc='upper right')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

#plt.figure()
#plt.histogram(men_master_student.loc[:,'Time to Finish'].values)

#print(women_master_student['Time to Finish'].values)


#print(women_master_student['Time to Finish'])
#women_master_student['Time to Finish'].hist(bins=20)
#men_master_student['Time to Finish'].hist(bins=20)
print(women_master_student['Time to Finish'].values.shape)
print(men_master_student['Time to Finish'].values.shape)

#data = np.vstack([women_master_student['Time to Finish'].values, men_master_student['Time to Finish'].values])

#plt.hist(women_master_student['Time to Finish'].values, alpha=0.5, bins=20, stacked=True)
#plt.hold()
#plt.hist(men_master_student['Time to Finish'].values,alpha=0.5, bins=20, stacked=True)


f, axarr = plt.subplots(2, sharex=True)
axarr[0].hist(women_master_student['Time to Finish'].values, alpha=0.5, bins=15, stacked=True, color='r')
axarr[0].set_title('Histogram plot of time taken to finish for each sex')
axarr[1].hist(men_master_student['Time to Finish'].values,alpha=0.5, bins=15, stacked=True, color='b')
axarr[0].set_ylabel('Number of \nfemale students')
axarr[1].set_ylabel('Number of \nmale students')
axarr[1].set_xlabel('Semesters taken to complete the master')
plt.show()

We indeed see that the distributions are the same, which agrees with the statistical test that gives no difference of duration beteween men and women.